# Deep Learning using Pytorch

In [1]:
import numpy as np
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn , optim
import torch.nn.functional as F
import warnings;warnings.filterwarnings('ignore');

In [2]:
# Transform the data to torch tensors and normalize it 
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307), ((0.3081)))])

# Shuffle the indices
indices = np.arange(60000)
np.random.shuffle(indices)

# Prepare the datasets
trainset = torchvision.datasets.MNIST('mnist', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST('mnist', train=False, download=True, transform=transform)

# Prepare the dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,shuffle=False,
                                          sampler=torch.utils.data.SubsetRandomSampler(indices[:55000]))
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,shuffle=False, num_workers=1)

valloader = torch.utils.data.DataLoader(trainset, batch_size=32,shuffle=False,
                                      sampler=torch.utils.data.SubsetRandomSampler(indices[55000:60000]))

testloader = torch.utils.data.DataLoader(testset, batch_size=32,shuffle=False, num_workers=1)     

In [3]:
# Compute the shape of the training set and testing set
testset_shape = testloader.dataset.test_data.shape
print(testset_shape)

# Compute the size of the minibatch for training set and testing set
testset_batchsize = testloader.batch_size
print(testset_batchsize)

torch.Size([10000, 28, 28])
32


In [4]:
# Define the class Net
class Net(nn.Module):
    def __init__(self):    
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):   
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model = Net()   
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
  
for i, data in enumerate(trainloader):
    X_train = data[0]
    y_train = data[1]
    
    X_train = X_train.view(-1, 28 * 28)
    optimizer.zero_grad()

    # Complete a forward pass
    output = model(X_train)

    # Compute the loss, gradients and change the weights
    loss = criterion(output,y_train)
    loss.backward()
    optimizer.step()

In [6]:
# Set the model in eval mode
model.eval()

correct, total = 0, 0

for i, data in enumerate(testloader, 0):
    X_test,y_test = data
    
    # Put each image into a vector
    X_test = X_test.view(-1,28*28)
    
    # Do the forward pass and get the predictions
    predictions = model(X_test)
    _, predictions = torch.max(predictions.data, 1)
    
    total += y_test.size(0)
    correct += (predictions == y_test).sum().item()
    
print('The testing set accuracy : %f %%' % (100 * correct / total))

The testing set accuracy : 95.110000 %


# Convulation Neural Network

In [7]:
class Net(nn.Module):
    def __init__(self, num_classes= 10):
        super(Net, self).__init__()
        
        # Instantiate the ReLU nonlinearity
        self.relu = nn.ReLU()
        
        # Instantiate two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3, padding=1)
        
        # Instantiate a max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Instantiate a fully connected layer
        self.fc = nn.Linear(7 * 7 * 40, num_classes)

    def forward(self, x):

        # Apply conv followd by relu, then in next line pool
        x = self.pool(self.relu(self.conv1(x)))

        # Apply conv followd by relu, then in next line pool
        x = self.pool(self.relu(self.conv2(x)))

        # Prepare the image for the fully connected layer
        x = x.view(-1, 7*7*40)

        # Apply the fully connected layer and return the result
        return self.fc(x)

In [8]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model_cnn = Net()   
optimizer = optim.Adam(model_cnn.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
  
for i, data in enumerate(trainloader,0):
    image_train,label_train = data
    optimizer.zero_grad()

    # Complete a forward pass
    output = model_cnn(image_train)

    # Compute the loss, gradients and change the weights
    loss = criterion(output,label_train)
    loss.backward()
    optimizer.step()

In [9]:
# Set the model in eval mode
model_cnn.eval()

correct, total = 0, 0

for i, data in enumerate(testloader, 0):
    image_test,label_test = data
    
    # Do the forward pass and get the predictions
    predictions = model_cnn(image_test)
    _, predictions = torch.max(predictions.data, 1)
    
    total += label_test.size(0)
    correct += (predictions == label_test).sum().item()
    
print('The testing set accuracy : %f %%' % (100 * correct / total))

The testing set accuracy : 97.180000 %


# Sequential CNN

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Declare all the layers for feature extraction
        self.features = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1), 
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1), 
                                      nn.MaxPool2d(2, 2), nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3, padding=1),
                                      nn.MaxPool2d(2, 2), nn.ReLU(inplace=True))
        
        # Declare all the layers for classification
        self.classifier = nn.Sequential(nn.Linear(7 * 7 * 40, 1024), nn.ReLU(inplace=True),
                                       	nn.Linear(1024, 2048), nn.ReLU(inplace=True),
                                        nn.Linear(2048, 10))
        
    def forward(self, x):
      
        # Apply the feature extractor in the input
        x = self.features(x)
        x = x.view(-1, 7 * 7 * 40)
        x = self.classifier(x)
        return x

In [11]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model_cnn = Net()   
optimizer = optim.Adam(model_cnn.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
  
for i, data in enumerate(trainloader,0):
    image_train,label_train = data
    optimizer.zero_grad()

    # Complete a forward pass
    output = model_cnn(image_train)

    # Compute the loss, gradients and change the weights
    loss = criterion(output,label_train)
    loss.backward()
    optimizer.step()
 

# model in evaluation mode
model_cnn.eval()

correct, total = 0, 0
for i, data in enumerate(testloader, 0):
    image_test,label_test = data
    
    # Do the forward pass and get the predictions
    predictions = model_cnn(image_test)
    _, predictions = torch.max(predictions.data, 1)
    
    total += label_test.size(0)
    correct += (predictions == label_test).sum().item()
    
print('The testing set accuracy : %f %%' % (100 * correct / total))

The testing set accuracy : 98.510000 %
